In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
import pynhanes

## User-defined dictionary of combined varibles

#### Normally, data analysis does not require all variables from each NHANES category. 

`pynhanes` expects user to provide a manually created dictionary of needed variables with human-readable names and corresponding combination of variable codes. This can be done in either two ways:

a) Hardcoded explicitly (see next cell), or

b) Provided in a .json file (see https://github.com/timpyrkov/pynhanes/blob/master/scripts/nhanes_variables.json)

Below we will use variant b), path to the .json file given in the `CSV` subfolder of the data folder.


In [ ]:
# """ Provide selected variables combinations explicitly: """
# variables = {
#     "Age":                                      ["RIDAGEYR"],
#     "Gender":                                   ["RIAGENDR"],
#     "Survey":                                   ["SDDSRVYR"],
#     "Weight (kg)":                              ["BMXWT"],
#     "Height (cm)":                              ["BMXHT"],
#     "BMI (kg/m2)":                              ["BMXBMI"],
#     "Mortality event":                          ["MORTSTAT"],
#     "Mortality tte":                            ["PERMTH_INT"],
#     "Health general":                           ["HSD010"],
#     "Smoking status":                           ["SMQ020", "SMQ120", "SMQ150"],
#     "Smoking regularly":                        ["SMD030", "SMD130", "SMD160"],
#     "Smoking now":                              ["SMQ040", "SMQ140", "SMQ170"],
# }

## Load userdata

#### Load data from default places:

- `nhanes_userdata.csv` and `nhanes_codebook.csv` in `CSV` subfolder

- `nhanes_counts.npz` and `nhanes_triax.npz` in `NPZ` subfolder


In [ ]:
""" Folder where parsed NHANES data are stored in 'CSV' and 'NPZ' subfolders """
datafolder = os.path.expanduser("~/work/NHANES/")

""" Read variables data """
""" Initialize codebook - an instance of pynhanes.CodeBook class """
codebook = pynhanes.CodeBook(variables=f"{datafolder}/CSV/nhanes_variables.json",
                             path_csv=f"{datafolder}/CSV/nhanes_codebook.csv")

""" Read user data """
""" Initialize nhanes - an instance of pynhanes.NhanesLoader class """
nhanes = pynhanes.NhanesLoader(path_npz=f"{datafolder}/NPZ/",
                                path_csv=f"{datafolder}/CSV/nhanes_userdata.csv")


## List available parsed data

#### `NhanesLoader` class has methods:
* `categories()` - list all loaded data categories
* `columns()` - list loaded data fields (columns) - either all or by category

#### `CodeBook` calss has method:
* `dict()` - list text decoding for numbered categorical values for a data field 
      (the output dictionary will be empty for a continuous-range data field)

In [ ]:
""" List all loaded data categories """
print("\nCATEGORIES:")
for categ in nhanes.categories():
    print(categ)

""" List data fileds (columns) for 'Demographic' category """
print("\nCOLUMNS:")
for col in nhanes.columns("Demographic"):
    print(col)
    """ 
    For each column list dictionary num -> text.
    Note that 'Age' dictionary lacks most of ages, this is because
    'Age' is a continuous-range data field with values ranging 5 - 80/85
    (keys '80' and '85' kept to notify you that though this is a continuous-rage data field,
    it is top-coded, i.e. ages >= 80 or 85 may be recorded as category named '80' or '85')
    """
    print(codebook.dict[col])

# """ List all data fileds (columns) """
# print("\nCOLUMNS:")
# for col in nhanes.columns():
#     print(col)


## Plot average and fraction of data values along lifespan

#### Useful utility functions help to gain quick impression of NHANES data distributions.

* age_avg_std() - outputs average and standard deviation of field values for each age
* age_fraction() - outputs fraction of selected field value for each age
<br>

* plot_age_avg_std() - plots average $\pm$ standard deviation of field values for each age
* plot_age_fraction() - plots fraction of field values for each age


In [ ]:
""" 
1. Plot average +/- std of self-reported general health status along lifespan.
 - We will use the averaging window = 3 [years] to produce a smooth line.
 - Note, that average value increases with age. This is because the data field
   is encoded from 1 - Excellent through 5 - Poor.
"""
field = "Health general"
age = nhanes.userdata("Age")
values = nhanes.userdata(field)
print(codebook.dict[field])

plt.figure(figsize=(8,4), facecolor="white")
plt.title(field)
pynhanes.plot_age_avg_std(values, age, window=3)
plt.xlabel("Age")
plt.ylabel("Average value")
plt.show()


""" 
2. Categorical data. Plot fraction of self-reported smoking status along lifespan.
"""

field = "Smoking status"
age = nhanes.userdata("Age")
values = nhanes.userdata(field)
labels = codebook.dict[field]

plt.figure(figsize=(8,4), facecolor="white")
plt.title(field)
pynhanes.plot_age_fraction(values, age, labels=labels)
plt.xlabel("Age")
plt.ylabel("Population fraction")
plt.show()


""" 
3. Continuous-range data. Plot fraction of body mass index along lifespan.
 - Data need to be digitized in either of the two (equivavlent) ways:
 - a) Using digitize() function (see commented code)
 - b) Automatically inside plot_age_fraction() if the number of unique values exceeds 15
 - In both cases the number of groups is given by the 'nbin' parameter
"""

field = "BMI (kg/m2)"
age = nhanes.userdata("Age")
values = nhanes.userdata(field)

plt.figure(figsize=(8,4), facecolor="white")
plt.title(field)
# values, labels = pynhanes.digitize(values, nbin=5)
# pynhanes.plot_age_fraction(values, age, labels=labels)
pynhanes.plot_age_fraction(values, age)
plt.xlabel("Age")
plt.ylabel("Population fraction")
plt.show()


## Statistical significance

#### One feature need to be taken into account when calculating statistical significance of assocaitions between health/lifestyle/demography factors is the number of samples.

P-value is a probability that the hypothesis that two distributions were different. 

 - We expect e.g. self-reported health is different for smokers and non-smokers
 - Lower p-value means lower probability that health is the same for these groups
 - P-value < 0.05 means significant difference for groups of size 100 - 1000 samples
 
 - Log p-value linearly scales down with the increasing number of samples
 - It is therefore incorrect to compare p-value for groups of 100/100 and 5000/5000 samples
 - Correct way can be to average log-pvalue over resamplings of the same size. This is 
   similar to Fisher’s combined probability method used for meta-studies [Fisher RA. 
   Statistical methods for research workers, (5th Ed). Oliver and Boyd, Edinburgh. 1934.]


In [ ]:
""" Get age, health, and smoking data """
age = nhanes.userdata("Age")
health = nhanes.userdata("Health general")
smoking = nhanes.userdata("Smoking status")

""" Convert smoking to text labels using codebook """
dct = codebook.dict["Smoking status"]
smoking = np.vectorize(dct.get)(smoking)

""" 
Create pandas DataFrame for age = 30 - 40 y.o. 
(We need pandas DataFrame to plot data using seaborn library)
"""
mask = (age >= 30) & (age < 40)
data = {"Age": age[mask], "Health": health[mask], "Smoking": smoking[mask]}
df = pd.DataFrame.from_dict(data)
df.dropna(inplace=True)

""" 
Draw a boxplot. (We will use enhanced boxplot, because general health data 
is categorical, and boxplot does not work well with categorical data)
"""
plt.figure(figsize=(8,4), facecolor="white")
plt.title("General health vs Smoking status (Age = 20 - 30)")
sns.boxenplot(x="Smoking", y="Health", data=df, order=np.unique(df["Smoking"].values))
plt.show()

"""
Statistical significance. Visually, 'current smokers' tend to have more higher values,
that is have more labels 'poor' health. However, straingforward calculation of p-values 
is not correct, becasue the number of samples in each group is very different: 
1676 never-smokers, 690 current smokers, and 424 of those who quit.
"""
print(df["Smoking"].value_counts())
print()

"""
We will use pynhanes utility function pvalue() to log-average p-values 
over several resamplings each of size 200 sample from each group.
Indeed, we see that health status of Current smokers is significantly (p < 5e-02)
different from both Never- and quit, while the latter two are not different (p > 5e-02).
"""
smoking = df["Smoking"].values.astype(str)
health = df["Health"].values
labels = np.unique(smoking)
for i, label1 in enumerate(labels):
    label2 = np.roll(labels, 1)[i]
    x1 = health[smoking == label1]
    x2 = health[smoking == label2]
    p = pynhanes.pvalue(x1, x2, n=200)
    print(f"{label1} / {label2}: p = {p:.2e}")


## Detrending (Adjustment) for age/sex

#### The other feature we need to take into account is that health outcomes depend on several confounding factors, including age/sex. Data should be adjusted for them before calculating statistical significance.



In [ ]:
""" Get age, health, and smoking data """
age = nhanes.userdata("Age")
health = nhanes.userdata("Health general")
smoking = nhanes.userdata("Smoking status")

""" 
More high self-reported values (more 'poor' health reports) 
are observed for both older ages and current smokers. 
"""
plt.figure(figsize=(8,4), facecolor="white")
plt.title("Self-reported general health")
cmap = plt.get_cmap("tab10")
for i in range(3):
    mask = smoking == i
    pynhanes.plot_age_avg_std(health[mask], age[mask], window=3, 
                              color=cmap(i), label=dct[i])
plt.xlabel("Age")
plt.ylabel("Average value")
plt.legend()
plt.show()

"""
Adjust (detrend) for age to stude the effect of smoking only.
"""
health_dtr = pynhanes.age_detrending(health, age, window=3)
plt.figure(figsize=(8,4), facecolor="white")
plt.title("Self-reported general health detrended by age")
cmap = plt.get_cmap("tab10")
for i in range(3):
    mask = smoking == i
    pynhanes.plot_age_avg_std(health_dtr[mask], age[mask], window=3, 
                              color=cmap(i), label=dct[i])
plt.xlabel("Age")
plt.ylabel("Average value")
plt.legend()
plt.show()

""" 
Now we repeat the code from the previous cell, but using 
the full age range and changing health to health_dtr.
"""
dct = codebook.dict["Smoking status"]
smoking = np.vectorize(dct.get)(smoking)

data = {"Age": age, "Health": health_dtr, "Smoking": smoking}
df = pd.DataFrame.from_dict(data)
df.dropna(inplace=True)

plt.figure(figsize=(8,4), facecolor="white")
plt.title("General health vs Smoking status")
sns.boxenplot(x="Smoking", y="Health", data=df, order=np.unique(df["Smoking"].values))
plt.show()

smoking = df["Smoking"].values.astype(str)
health = df["Health"].values
labels = np.unique(smoking)
for i, label1 in enumerate(labels):
    label2 = np.roll(labels, 1)[i]
    x1 = health[smoking == label1]
    x2 = health[smoking == label2]
    p = pynhanes.pvalue(x1, x2, n=200)
    print(f"{label1} / {label2}: p = {p:.2e}")
